# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from vacapy_api import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load in file
vaca_start = "../WeatherPy/city_summary_df.csv"


In [3]:
#read and display csv
vaca_start_df = pd.read_csv(vaca_start)
vaca_start_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,east london,-33.0153,27.9116,74.59,88,6,7.27,ZA,1612652802
1,atuona,-9.8000,-139.0333,80.28,77,89,21.81,PF,1612652567
2,sibu,2.3000,111.8167,75.20,100,40,1.63,MY,1612652819
3,havre-saint-pierre,50.2334,-63.5986,28.40,100,90,16.11,CA,1612652848
4,college,64.8569,-147.8028,-6.00,77,90,0.31,US,1612652569
...,...,...,...,...,...,...,...,...,...
586,bulgan,48.8125,103.5347,1.78,83,5,2.75,MN,1612652824
587,vila,42.0304,-8.1588,39.00,88,53,3.51,PT,1612652830
588,santa maria,-29.6842,-53.8069,75.20,69,0,3.44,BR,1612652623
589,tsabong,-26.0217,22.4010,70.86,76,0,15.93,BW,1612652875


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#store locations
locations = vaca_start_df[["Lat", "Lng"]]
#store humidity
humidity = vaca_start_df['Humidity']

In [5]:
#plot heatmap
fig = gmaps.figure()

#create heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
#add layer
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

max_temp_df = vaca_start_df[(vaca_start_df['Max Temp']> 75) & (vaca_start_df['Max Temp']< 85)]
max_temp_df
wind_choice_df = vaca_start_df[vaca_start_df['Wind Speed'] <5]
wind_choice_df
cloud_choice_df = vaca_start_df[vaca_start_df['Cloudiness'] <10]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [85]:
#merge all 3 where there is overlap to make hotel_df
city_summary_2_df = pd.merge(max_temp_df, wind_choice_df, how = "inner", on="City")
hotel_df= pd.merge (city_summary_2_df, cloud_choice_df, how = "inner", on="City")
#hotel_df
hotel_df["hotel name"] = ''
hotel_df = hotel_df.drop('Lat_x', 1)
hotel_df = hotel_df.drop('Lng_x', 1)
hotel_df = hotel_df.drop('Max Temp_x', 1)
hotel_df = hotel_df.drop('Humidity_x', 1)
hotel_df = hotel_df.drop('Cloudiness_x', 1)
hotel_df = hotel_df.drop('Wind Speed_x', 1)
hotel_df = hotel_df.drop('Country_x', 1)
hotel_df = hotel_df.drop('Date_x', 1)
hotel_df = hotel_df.drop('Lat_y', 1)
hotel_df = hotel_df.drop('Lng_y', 1)
hotel_df = hotel_df.drop('Max Temp_y', 1)
hotel_df = hotel_df.drop('Humidity_y', 1)
hotel_df = hotel_df.drop('Cloudiness_y', 1)
hotel_df = hotel_df.drop('Wind Speed_y', 1)
hotel_df = hotel_df.drop('Country_y', 1)
hotel_df = hotel_df.drop('Date_y', 1)
hotel_df
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel name
0,mercedes,-34.6515,-59.4307,80.01,72,0,1.19,AR,1612652818,
1,grand gaube,-20.0064,57.6608,77.00,85,8,1.99,MU,1612652579,
2,keffi,8.8486,7.8736,78.80,41,2,3.44,NG,1612652674,
3,vao,-22.6667,167.4833,77.86,81,0,4.09,NC,1612652373,
4,nueva helvecia,-34.3000,-57.2333,84.20,48,0,4.61,UY,1612652860,
5,koumac,-20.5667,164.2833,82.80,61,0,1.25,NC,1612652862,
6,pouebo,-20.4000,164.5667,83.35,60,1,1.41,NC,1612652864,
7,santarem,-2.4431,-54.7083,78.80,83,0,3.76,BR,1612652868,
8,monsenhor gil,-5.5642,-42.6078,84.20,65,0,3.44,BR,1612652868,
9,tadine,-21.5500,167.8833,81.93,70,0,2.86,NC,1612652871,


In [37]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel type from df
    hotel_type = row['hotel name']

    # add keyword to params dict
    params['keyword'] = hotel_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {hotel_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {hotel_type} hotel is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: .
Missing field/result... skipping.
------------
Retrieving Results for Index 1: .
Missing field/result... skipping.
------------
Retrieving Results for Index 2: .
Missing field/result... skipping.
------------
Retrieving Results for Index 3: .
Missing field/result... skipping.
------------
Retrieving Results for Index 4: .
Missing field/result... skipping.
------------
Retrieving Results for Index 5: .
Missing field/result... skipping.
------------
Retrieving Results for Index 6: .
Missing field/result... skipping.
------------
Retrieving Results for Index 7: .
Missing field/result... skipping.
------------
Retrieving Results for Index 8: .
Missing field/result... skipping.
------------
Retrieving Results for Index 9: .
Missing field/result... skipping.
------------
Retrieving Results for Index 10: .
Missing field/result... skipping.
------------
Retrieving Results for Index 11: .
Missing field/result... skipping.
------------
Retrieving Results for Ind

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map


# Display figure

